# Modelos de clasificación curricular

Los siguientes modelos machine learning permitirán obtener validar una carga curricular personalizada de acuerdo a variables relacionadas con el rendimiento escolar del estudiante y variables relacionadas con la dificultad general de las asignaturas.

In [1]:
import pandas as pd
import numpy as np

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [22]:
# Importar todos los datos
datos_2016 = pd.read_csv("datos_2016.csv")
datos_2017 = pd.read_csv("datos_2017.csv")
datos_2018 = pd.read_csv("datos_2018.csv")
datos_2019 = pd.read_csv("datos_2019.csv")
datos_2020_2021 = pd.read_csv("2020_2021.csv")

In [23]:
datos_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39354 entries, 0 to 39353
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          39354 non-null  int64  
 1   carrera             39354 non-null  object 
 2   periodo             39354 non-null  int64  
 3   clave               39354 non-null  object 
 4   asignatura          39354 non-null  object 
 5   promediofinal       39354 non-null  float64
 6   docente             39354 non-null  object 
 7   ceveval_global      38463 non-null  float64
 8   ceneval_analitico   38463 non-null  float64
 9   ceneval_matematico  38463 non-null  float64
 10  ceneval_lengua      38463 non-null  float64
 11  ceneval_esp         38463 non-null  float64
 12  modulo1             38463 non-null  object 
 13  r_modulo1           38463 non-null  object 
 14  modulo2             38463 non-null  object 
 15  r_modulo2           38463 non-null  object 
 16  modu

In [24]:
datos_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41540 entries, 0 to 41539
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          41540 non-null  int64  
 1   carrera             41540 non-null  object 
 2   periodo             41540 non-null  int64  
 3   clave               41540 non-null  object 
 4   asignatura          41540 non-null  object 
 5   promediofinal       41539 non-null  float64
 6   docente             41540 non-null  object 
 7   ceveval_global      40660 non-null  float64
 8   ceneval_analitico   40660 non-null  float64
 9   ceneval_matematico  40660 non-null  float64
 10  ceneval_lengua      40660 non-null  float64
 11  ceneval_esp         40660 non-null  float64
 12  modulo1             40660 non-null  object 
 13  r_modulo1           40660 non-null  object 
 14  modulo2             40660 non-null  object 
 15  r_modulo2           40660 non-null  object 
 16  modu

In [25]:
datos_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          42000 non-null  int64  
 1   carrera             42000 non-null  object 
 2   periodo             42000 non-null  int64  
 3   clave               42000 non-null  object 
 4   asignatura          42000 non-null  object 
 5   promediofinal       41984 non-null  float64
 6   docente             42000 non-null  object 
 7   ceveval_global      41248 non-null  float64
 8   ceneval_analitico   41248 non-null  float64
 9   ceneval_matematico  41248 non-null  float64
 10  ceneval_lengua      41248 non-null  float64
 11  ceneval_esp         41248 non-null  float64
 12  modulo1             41248 non-null  object 
 13  r_modulo1           41248 non-null  object 
 14  modulo2             41248 non-null  object 
 15  r_modulo2           41248 non-null  object 
 16  modu

In [26]:
datos_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44787 entries, 0 to 44786
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          44787 non-null  int64  
 1   carrera             44787 non-null  object 
 2   periodo             44787 non-null  int64  
 3   clave               44787 non-null  object 
 4   asignatura          44787 non-null  object 
 5   promediofinal       44787 non-null  float64
 6   docente             44787 non-null  object 
 7   ceveval_global      44080 non-null  float64
 8   ceneval_analitico   44080 non-null  float64
 9   ceneval_matematico  44080 non-null  float64
 10  ceneval_lengua      44080 non-null  float64
 11  ceneval_esp         44080 non-null  float64
 12  modulo1             44080 non-null  object 
 13  r_modulo1           44080 non-null  object 
 14  modulo2             44080 non-null  object 
 15  r_modulo2           44080 non-null  object 
 16  modu

In [27]:
datos_2020_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9947 entries, 0 to 9946
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      9947 non-null   int64  
 1   Column1         9947 non-null   int64  
 2   programa        9947 non-null   object 
 3   periodo         9947 non-null   int64  
 4   clave           9947 non-null   object 
 5   asignatura      9947 non-null   object 
 6   docente         9947 non-null   object 
 7   promediofinal   9887 non-null   float64
 8   ceveval_global  9453 non-null   float64
 9   hrs_trabaja     9451 non-null   object 
 10  n_matricula     9947 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 854.9+ KB


In [28]:
# Consolidar toda la información en un mismo dataframe
data = pd.concat([datos_2016, datos_2017, datos_2018, datos_2019, datos_2020_2021], axis = 0, ignore_index = True)

In [29]:
data.head()

,Unnamed: 0,carrera,periodo,clave,asignatura,promediofinal,docente,ceveval_global,ceneval_analitico,ceneval_matematico,ceneval_lengua,ceneval_esp,modulo1,r_modulo1,modulo2,r_modulo2,modulo3,r_modulo3,modulo4,r_modulo4,modulo5,r_modulo5,modulo6,r_modulo6,modulo7,r_modulo7,modulo8,r_modulo8,modulo9,r_modulo9,modulo10,r_modulo10,modulo11,r_modulo11,modulo12,r_modulo12,n_matricula,Column1,programa,hrs_trabaja
0,0,Negocios Internacionales,201601,DP0295,Taller de formación en responsabilidad social ...,10.0,Maldonado Saldaña / Gisela,1150.0,1108.0,1228.0,1108.0,1156.0,México: Geografía e Historia,1300,Español Superior y Literatura,1300,Humanidades,1300,Derecho y Ciencias Sociales,1300,Matemáticas,-,Física,-,Química,-,Biología Humana y Ciencias de la Salud,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285,NaN,NaN,NaN
1,1,Negocios Internacionales,201601,NI0215,Taller de software para negocios internacionales,9.0,Villeda Cuellar / Víctor Hugo,1150.0,1108.0,1228.0,1108.0,1156.0,México: Geografía e Historia,1300,Español Superior y Literatura,1300,Humanidades,1300,Derecho y Ciencias Sociales,1300,Matemáticas,-,Física,-,Química,-,Biología Humana y Ciencias de la Salud,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285,NaN,NaN,NaN
2,2,Negocios Internacionales,201601,LI1104,Nivel 4 Inglés,10.0,Ross / Andrew Patrick Simon,1150.0,1108.0,1228.0,1108.0,1156.0,México: Geografía e Historia,1300,Español Superior y Literatura,1300,Humanidades,1300,Derecho y Ciencias Sociales,1300,Matemáticas,-,Física,-,Química,-,Biología Humana y Ciencias de la Salud,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285,NaN,NaN,NaN
3,3,Negocios Internacionales,201601,NI0209,Administración financiera,6.0,Vallejo Filoteo / Jorge,1150.0,1108.0,1228.0,1108.0,1156.0,México: Geografía e Historia,1300,Español Superior y Literatura,1300,Humanidades,1300,Derecho y Ciencias Sociales,1300,Matemáticas,-,Física,-,Química,-,Biología Humana y Ciencias de la Salud,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285,NaN,NaN,NaN
4,4,Negocios Internacionales,201601,NI0213,Transporte y logística en comercio exterior,9.0,Cañedo Magaña / Magdalena del Carmen,1150.0,1108.0,1228.0,1108.0,1156.0,México: Geografía e Historia,1300,Español Superior y Literatura,1300,Humanidades,1300,Derecho y Ciencias Sociales,1300,Matemáticas,-,Física,-,Química,-,Biología Humana y Ciencias de la Salud,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285,NaN,NaN,NaN


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177628 entries, 0 to 177627
Data columns (total 40 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          177628 non-null  int64  
 1   carrera             167681 non-null  object 
 2   periodo             177628 non-null  int64  
 3   clave               177628 non-null  object 
 4   asignatura          177628 non-null  object 
 5   promediofinal       177551 non-null  float64
 6   docente             177628 non-null  object 
 7   ceveval_global      173904 non-null  float64
 8   ceneval_analitico   164451 non-null  float64
 9   ceneval_matematico  164451 non-null  float64
 10  ceneval_lengua      164451 non-null  float64
 11  ceneval_esp         164451 non-null  float64
 12  modulo1             164451 non-null  object 
 13  r_modulo1           164451 non-null  object 
 14  modulo2             164451 non-null  object 
 15  r_modulo2           164451 non-nul

In [31]:
# Eliminar la primera columa de la izquierda
data.drop(columns = ["Unnamed: 0", "Column1", "programa"], inplace = True)

In [32]:
data.columns

Index(['carrera', 'periodo', 'clave', 'asignatura', 'promediofinal', 'docente',
       'ceveval_global', 'ceneval_analitico', 'ceneval_matematico',
       'ceneval_lengua', 'ceneval_esp', 'modulo1', 'r_modulo1', 'modulo2',
       'r_modulo2', 'modulo3', 'r_modulo3', 'modulo4', 'r_modulo4', 'modulo5',
       'r_modulo5', 'modulo6', 'r_modulo6', 'modulo7', 'r_modulo7', 'modulo8',
       'r_modulo8', 'modulo9', 'r_modulo9', 'modulo10', 'r_modulo10',
       'modulo11', 'r_modulo11', 'modulo12', 'r_modulo12', 'n_matricula',
       'hrs_trabaja'],
      dtype='object')

In [33]:
# Renombrar columnas a un estándar en Inglés
data.rename(columns = {
    "carrera" : "career", 
    "periodo" : "period", 
    "clave" : "id_subject",
    "asignatura" : "subject", 
    "promediofinal" : "final_grade",
    "docente" : "teacher",
    "ceveval_global" : "global",
    "ceneval_analitico" : "analytic",
    "ceneval_matematico" : "math",
    "ceneval_lengua" : "language",
    "ceneval_esp" : "spanish",
    "n_matricula" : "id_user",
    "hrs_trabaja" : "working_hours"
}, inplace = True)

In [34]:
# Filtrar los nombres de los módulos
modulos = data.columns[data.columns.str.contains("modulo")]
modulos

Index(['modulo1', 'r_modulo1', 'modulo2', 'r_modulo2', 'modulo3', 'r_modulo3',
       'modulo4', 'r_modulo4', 'modulo5', 'r_modulo5', 'modulo6', 'r_modulo6',
       'modulo7', 'r_modulo7', 'modulo8', 'r_modulo8', 'modulo9', 'r_modulo9',
       'modulo10', 'r_modulo10', 'modulo11', 'r_modulo11', 'modulo12',
       'r_modulo12'],
      dtype='object')

In [35]:
# Eliminar las columnas de los módulos
data.drop(columns = modulos, inplace = True)

data.columns

Index(['career', 'period', 'id_subject', 'subject', 'final_grade', 'teacher',
       'global', 'analytic', 'math', 'language', 'spanish', 'id_user',
       'working_hours'],
      dtype='object')

In [36]:
# Revisar filas duplicadas
data[data.duplicated(keep = False)].head()

,career,period,id_subject,subject,final_grade,teacher,global,analytic,math,language,spanish,id_user,working_hours
405,Ingeniería en Datos e Inteligencia Organizacional,201601,IT0107,Técnicas algorítmicas,9.0,Flores Granados / David Israel,1138.0,1132.0,1180.0,1132.0,1108.0,140311177,NaN
406,Ingeniería en Datos e Inteligencia Organizacional,201601,IT0107,Técnicas algorítmicas,9.0,Flores Granados / David Israel,1138.0,1132.0,1180.0,1132.0,1108.0,140311177,NaN
1267,Turismo Sustentable y Gestión Hotelera,201601,TS0102,Introducción a la hotelería,0.0,Hernández Méndez / Juan Oscar,1000.0,1084.0,1012.0,988.0,916.0,150311068,NaN
1275,Turismo Sustentable y Gestión Hotelera,201601,TS0102,Introducción a la hotelería,0.0,Hernández Méndez / Juan Oscar,1000.0,1084.0,1012.0,988.0,916.0,150311068,NaN
15027,Ingeniería en Datos e Inteligencia Organizacional,201603,IT0107,Técnicas algorítmicas,7.0,Manzano Pinzón / Francisco,1198.0,1180.0,1276.0,1156.0,1180.0,150311223,NaN


In [37]:
# Eliminar registros duplicados
data.drop_duplicates(keep = "first", inplace = True)

In [38]:
# Esta vez no habrán duplicados
data[data.duplicated(keep = False)].head()

,career,period,id_subject,subject,final_grade,teacher,global,analytic,math,language,spanish,id_user,working_hours


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177613 entries, 0 to 177627
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   career         167666 non-null  object 
 1   period         177613 non-null  int64  
 2   id_subject     177613 non-null  object 
 3   subject        177613 non-null  object 
 4   final_grade    177536 non-null  float64
 5   teacher        177613 non-null  object 
 6   global         173890 non-null  float64
 7   analytic       164437 non-null  float64
 8   math           164437 non-null  float64
 9   language       164437 non-null  float64
 10  spanish        164437 non-null  float64
 11  id_user        177613 non-null  int64  
 12  working_hours  9451 non-null    object 
dtypes: float64(6), int64(2), object(5)
memory usage: 19.0+ MB


In [44]:
grades_per_period = data.groupby(["career", "id_user", "period"]).size().reset_index(name = "n")
grades_per_period.sort_values(by = ["career", "id_user", "period", "n"], inplace = True)
grades_per_period

,career,id_user,period,n
0,Gastronomía,100311361,201601,9
1,Gastronomía,100311361,201603,8
2,Gastronomía,100311361,201604,1
3,Gastronomía,110311014,201601,1
4,Gastronomía,110311014,201602,1
...,...,...,...,...
31960,Turismo Sustentable y Gestión Hotelera,190311672,201903,4
31961,Turismo Sustentable y Gestión Hotelera,190311673,201903,6
31962,Turismo Sustentable y Gestión Hotelera,190311674,201903,6
31963,Turismo Sustentable y Gestión Hotelera,190311678,201903,4


In [48]:
# Conteo de agrupaciones por cantidad de asignaturas
grades_sizes = grades_per_period.groupby("n").size().reset_index(name = "count")
grades_sizes.sort_values(by = "count", ascending = False, inplace = True)
grades_sizes

,n,count
0,1,5785
6,7,4699
7,8,4594
8,9,4467
5,6,3597
2,3,2790
4,5,2337
1,2,2300
3,4,1367
9,10,22


In [51]:
# Promedio de calificaciones cargadas por periodo
grades_sizes["n"].mean()

7.615384615384615